In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras-tuner

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import cv2
import seaborn as sns
from datetime import datetime, timedelta
from google.cloud import storage

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import (confusion_matrix,accuracy_score)
from sklearn.metrics import classification_report

from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from keras_tuner.tuners import BayesianOptimization
from keras_tuner import HyperModel

In [ ]:
PROJECT='My First Project'
BUCKET='data_meta'
FILENAME='asl_alphabet_train/A/A1.jpg'

client = storage.Client(PROJECT)
bucket = client.get_bucket(BUCKET)

In [ ]:
start_time = datetime.now()

X = []
y = []
folders =  ['A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I','J', 'K', 'L', 'M', 'N' 
            ,'O', 'P', 'Q', 'R','S', 'T','U', 'V', 'W', 'X', 'Y', 'Z']

for label in folders:
    for blob in client.list_blobs(BUCKET, prefix='asl_alphabet_train/'+label):
        try:
            image = np.asarray(bytearray(blob.download_as_string()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)
            X.append(image)
            y.append(folders.index(label) + 1)
        except:
            pass
    print(label,':',len(X)-3000*(ord(label)-65))
    
end_time = datetime.now()

In [ ]:
time_taken = (end_time - start_time).total_seconds()/60
print(time_taken)

In [ ]:
np.save('input_array_full.npy',X)
np.save('label_array_full.npy',y)

In [ ]:
X = np.load('input_array.npy')
y = np.load('label_array.npy')

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
print(X.shape)
y.shape

In [ ]:
Onehot_encoding = OneHotEncoder()
y= np.reshape(y,(-1,1))
Onehot_encoding.fit(y)
y = Onehot_encoding.transform(y).toarray()

In [ ]:
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.7, random_state=42, stratify = y)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5, random_state=42, stratify = y_rem)

In [ ]:
del X,y

In [ ]:
X_train = preprocess_input(X_train)
X_valid = preprocess_input(X_valid)
X_test = preprocess_input(X_test)

In [ ]:
class tuner_inception_model(HyperModel):

    def __init__(self, num_classes=26):
        self.num_classes = num_classes

    def build(self, hp):

        inception_backbone = InceptionV3(weights='imagenet', include_top=False)

        x = inception_backbone.output
        x = GlobalAveragePooling2D()(x)

        x = Dense(1024, activation=hp.Choice(
                    'dense_activation',
                    values=['relu','sigmoid','tanh'],
                    default='relu'))(x)

        predictions = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=inception_backbone.input, outputs=predictions)

        for layer in inception_backbone.layers:
            layer.trainable = False

        model.compile(optimizer=keras.optimizers.RMSprop(
                hp.Choice(
                    'learning_rate',
                    values = [0.001,0.005,0.01,0.05,0.1],
                    default=1e-2)),
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

        return model

In [ ]:
class MyTuner(BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['batch_size'] = trial.hyperparameters.Choice('batch_size',values=[64,128,256,512],default = 256)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)

In [ ]:
tuned_model = tuner_inception_model()

In [ ]:
tuner = MyTuner(
    tuned_model,
    max_trials = 30,
    objective='val_accuracy',
    seed=42,
    executions_per_trial=1,
    directory='BayesianOptimization',
    project_name='ASL_recg_fin_incep'
)

In [ ]:
tuner.search_space_summary()

In [ ]:
start_run = datetime.now()
tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))
end_run = datetime.now()

In [ ]:
time_taken = (end_run - start_run).total_seconds()/60
print(time_taken)

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
print(best_model.history)

In [ ]:
loss_train = best_model.history['train_loss']
loss_val = best_model.history['val_loss']
epochs = range(1,100)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
best_model.save('my_model_100')

In [ ]:
prediction = best_model.predict(X_test)

In [ ]:
prediction_decoded = np.argmax(prediction,axis = 1)
y_test_decoded = np.argmax(y_test,axis = 1)

In [ ]:
cf_matrix = confusion_matrix(y_test_decoded,prediction_decoded)
sns.heatmap(cf_matrix,cmap = "YlGnBu")

In [ ]:
print(classification_report(y_test_decoded, prediction_decoded, target_names= [str(i) for i in range(1,27)]))

In [ ]:
print('accuracy:',accuracy_score(y_test_decoded, prediction_decoded))